In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

In [62]:
train_data_fpath =  "training_dataset.parquet"
submission_data_fpath =  "submission_dataset.parquet"

In [63]:
input_df = pd.read_parquet(train_data_fpath)
test_df = pd.read_parquet(submission_data_fpath)
input_df.head(3)

,TimeStamp_StartFormat,wtc_AcWindSp_mean;1,wtc_AcWindSp_mean;2,wtc_AcWindSp_mean;3,wtc_AcWindSp_mean;4,wtc_AcWindSp_mean;5,wtc_AcWindSp_mean;7,wtc_AcWindSp_min;1,wtc_AcWindSp_min;2,wtc_AcWindSp_min;3,...,ERA5_surface_pressure,ERA5_cloud_cover,ERA5_wind_speed_10m,ERA5_wind_speed_100m,ERA5_wind_direction_10m,ERA5_wind_direction_100m,ERA5_wind_gusts_10m,id,is_valid,target
0,2016-01-01 00:00:00,6.085917,5.830675,6.613091,6.998510,7.645727,7.040790,2.0,1.1,1.6,...,976.166321,0.0,6.080296,10.040418,233.695404,236.784729,13.0,-210384,True,352.522308
1,2016-01-01 00:10:00,5.824693,5.810768,5.895642,6.521900,6.413868,6.599652,2.1,2.2,1.9,...,982.179504,100.0,3.301515,6.958448,178.264328,187.431320,6.9,-210383,True,294.078888
2,2016-01-01 00:20:00,7.100180,6.386981,7.606015,7.695034,7.812548,7.469052,2.2,2.1,0.0,...,985.240540,78.0,3.478505,4.648656,198.435043,198.824799,7.7,-210382,True,587.302795


In [69]:
input_df.tail(3)

,TimeStamp_StartFormat,wtc_AcWindSp_mean;1,wtc_AcWindSp_mean;2,wtc_AcWindSp_mean;3,wtc_AcWindSp_mean;4,wtc_AcWindSp_mean;5,wtc_AcWindSp_mean;7,wtc_AcWindSp_min;1,wtc_AcWindSp_min;2,wtc_AcWindSp_min;3,...,ERA5_surface_pressure,ERA5_cloud_cover,ERA5_wind_speed_10m,ERA5_wind_speed_100m,ERA5_wind_direction_10m,ERA5_wind_direction_100m,ERA5_wind_gusts_10m,id,is_valid,target
210381,2019-12-31 23:30:00,8.292965,7.242282,7.387715,8.898215,7.778623,7.802809,4.98,4.98,4.49,...,996.174988,55.0,4.701064,9.276314,218.088821,221.941223,8.8,-3,True,687.353088
210382,2019-12-31 23:40:00,9.617872,9.914719,8.692904,10.193710,10.449450,8.951912,6.63,5.90,5.90,...,1001.884033,69.0,3.088689,6.236986,209.054504,221.099411,5.6,-2,True,1107.057007
210383,2019-12-31 23:50:00,9.997997,10.407530,9.129176,10.606570,10.896760,9.398952,7.26,5.70,6.11,...,1000.020325,89.0,3.008322,5.950631,195.422226,204.842361,4.0,-1,True,1328.432983


In [64]:
input_df= input_df[~input_df.target.isna()]

In [66]:
y = input_df["target"]
X = input_df[test_df.columns].drop(["TimeStamp_StartFormat"],axis=1)
X_test = test_df.drop(["TimeStamp_StartFormat"],axis=1)

In [67]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=123
)

In [68]:
xgb_model = xgb.XGBRegressor(
    objective='reg:absoluteerror',#absoluteerror#squarederror
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=10,#6
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
    random_state=42,
    n_jobs=-1
)

# Train the model
xgb_model.fit(#xgb_model
    X_train, 
    y_train,
    eval_set=[(X_val, y_val)],
    
    verbose=True
)

# Evaluate on validation set
val_pred = xgb_model.predict(X_val)
val_mae = mean_absolute_error(y_val, val_pred)
print(f"Validation MAE: {val_mae:.4f}")


# Make predictions
test_pred = xgb_model.predict(X_test)

# Save predictions
np.savetxt("predictions_xgboost.csv", test_pred, delimiter=",")

[0]	validation_0-mae:520.22229
[1]	validation_0-mae:495.35822
[2]	validation_0-mae:471.87349
[3]	validation_0-mae:449.16920
[4]	validation_0-mae:429.18938
[5]	validation_0-mae:408.23047
[6]	validation_0-mae:388.25843
[7]	validation_0-mae:369.53378
[8]	validation_0-mae:351.47040
[9]	validation_0-mae:334.32606
[10]	validation_0-mae:318.09902
[11]	validation_0-mae:303.28624
[12]	validation_0-mae:288.61828
[13]	validation_0-mae:274.62743
[14]	validation_0-mae:261.32544
[15]	validation_0-mae:249.53120
[16]	validation_0-mae:238.30332
[17]	validation_0-mae:227.57777
[18]	validation_0-mae:217.69674
[19]	validation_0-mae:208.35954
[20]	validation_0-mae:199.22125
[21]	validation_0-mae:190.57476
[22]	validation_0-mae:182.51696
[23]	validation_0-mae:174.98990
[24]	validation_0-mae:168.14383
[25]	validation_0-mae:161.44931
[26]	validation_0-mae:155.17397
[27]	validation_0-mae:149.45659
[28]	validation_0-mae:143.90005
[29]	validation_0-mae:138.67500
[30]	validation_0-mae:133.74705
[31]	validation_0-